In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["train/cats", "train/dogs"]  # Uso de barras normales
imgs = []  # Lista para almacenar las imágenes
labels = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 1)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs = np.array(imgs)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (557, 150, 150, 3)
Num. of labels: 557


In [3]:
# Crear un modelo secuencial correctamente
model = keras.Sequential([
    keras.layers.Input((150, 150, 3)), # Los dos primeros don las dimensiones, el último es la escala de color
    keras.layers.Conv2D(32,4, padding="same", strides=1, activation="relu"), # Empiricamente el profe dice que esta mejor comenzar con 32
    # El padding es para igualar el tamaño de la imágen de salida al de la imágen de entrada
    # El strinde de 1 es recomendable para clasificación
    keras.layers.MaxPooling2D(), # El valor del producto de cada matriz por el filtro que salga tomamos el mas grande
    keras.layers.Conv2D(16,4, padding="same", strides=2),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(8,2, padding="same", strides=1, activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.Dense(300),
    keras.layers.Dense(1, activation="sigmoid")
])

# Imprimir el resumen del modelo
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 38, 38, 16)     │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 19, 19, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 19, 19, 8)      │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2888)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │       866,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 300)            │        90,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           301 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 967,597 (3.69 MB)

 Trainable params: 967,597 (3.69 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
history = model.fit(imgs, labels, epochs=50, batch_size=32)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.5415 - loss: 0.9341
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5484 - loss: 0.6886
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.6838 - loss: 0.6649
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.6173 - loss: 0.6883
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.6419 - loss: 0.6313
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.7247 - loss: 0.5284
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8692 - loss: 0.3561
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9179 - loss: 0.2093
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9631 - loss: 0.1048
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9823 - loss: 0.0559
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9950 - loss: 0.0257
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy:

In [5]:
paths = ["test/cats", "test/dogs"]  # Uso de barras normales
imgs_test = []  # Lista para almacenar las imágenes
labels_test = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 1)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs_test.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels_test.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs_test = np.array(imgs_test)
labels_test = np.array(labels_test)

print(f"Images shape: {imgs_test.shape}")
print(f"Num. of labels_test: {len(labels_test)}")

Images shape: (140, 150, 150, 3)
Num. of labels: 140


In [6]:
predictions = model.predict(imgs_test)
predictions = np.argmax(predictions, axis = 1)
print(((predictions == labels_test).sum()/len(labels_test))*100)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
50.0
